<a href="https://colab.research.google.com/github/mansueli/Supa-Migrate/blob/main/Migrate_Project_%26_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Supabase](https://raw.githubusercontent.com/supabase/supabase/master/packages/common/assets/images/supabase-logo-wordmark--light.svg)



## Setting the Environment variables:

In [ ]:
!pip install python-dotenv &>log

import os
from dotenv import load_dotenv

load_dotenv()
%load_ext dotenv
%dotenv

foodshare_project_ref = os.getenv('foodshare_project_ref')
postgres_project_ref = os.getenv('postgres_project_ref')
supabase_foodshare_pass = os.getenv('supabase_foodshare_pass')
supabase_postgres_pass = os.getenv('supabase_postgres_pass')
foodshare_service_key = os.getenv('foodshare_service_key')
postgres_service_key = os.getenv('postgres_service_key')

OLD_DB_URL='https://foodshare_project_ref.supabase.co'
NEW_DB_URL='https://postgres_project_ref.supabase.co'
OLD_DB_PASS='supabase_foodshare_pass'
NEW_DB_PASS='postgres_foodshare_pass'
OLD_SERVICE_KEY='foodshare_service_key'
NEW_SERVICE_KEY='postgres_service_key'

: 

## Installing PSQL, Downloading the scripts:

In [ ]:
!sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!sudo apt-get update &>log
!sudo apt-get -y install postgresql &>log
!wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/migrate_project.sh &>log
!pip install supabase &>log

OK


## Running migration & moving storage objects:

In [ ]:
from supabase import create_client
import os
filedata = ''
with open('migrate_project.sh', 'r') as file :
  filedata = file.read()
  filedata = filedata.replace('db.old_project_ref.supabase.co', OLD_DB_URL.replace("https://","db."))
  filedata = filedata.replace('db.new_project_ref.supabase.co', NEW_DB_URL.replace("https://","db."))
  filedata = filedata.replace('secret_password_here', OLD_DB_PASS)
  filedata = filedata.replace('secret_new_password_here', NEW_DB_PASS)

with open('migrate_project.sh', 'w') as file:
  file.write(filedata)

#Run the database migration script:
!bash ./migrate_project.sh &>log

#creating the clients for the old & new projects
old_supabase_client = create_client(OLD_DB_URL, OLD_SERVICE_KEY)
new_supabase_client = create_client(NEW_DB_URL, NEW_SERVICE_KEY)

#Create all buckets
buckets = old_supabase_client.storage().list_buckets()
for bucket in buckets:
    print("Copying objects from "+bucket.name)
    objects = old_supabase_client.storage().from_(bucket.name).list()
    try:
      new_supabase_client.storage().create_bucket(bucket.name, public=bucket.public)
    except:
      print("unable to create bucket")
    for obj in objects:
        print(obj['name'])
        try:
          with open(obj['name'], 'wb+') as f:
            res = old_supabase_client.storage().from_(bucket.name).download(obj['name'])
            f.write(res)
            f.close()
        except Exception as e: 
            print("error downloading "+ str(e))
        try:
          with open(obj['name'], 'rb+') as f:
            res = new_supabase_client.storage().from_(bucket.name).upload(obj['name'], obj['name'])
          # Delete file after uploading it
          if os.path.exists(os.path.abspath(obj['name'])):
              os.remove(os.path.abspath(obj['name']))
        except Exception as e: 
          print("error uploading | " + str(e))